In [1]:
import pandas as pd
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'
    
dfs = pd.read_csv("final.csv")

In [2]:
df1 =dfs

In [3]:
df1 = df1.dropna()

In [4]:
df = df1.groupby('tag').agg({"pattern": lambda x:list(x),
                       "response" : lambda x: list(x)
                       })

In [5]:
df = df.reset_index(level=0)

In [6]:
corpus = df.to_dict("records")

In [7]:
data = {}
data['intents'] = corpus

In [8]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /Users/chenqian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/chenqian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()
# Each list to create
words = []
classes = []
doc_X = []
doc_y = []
# Loop through all the intents
# tokenize each pattern and append tokens to words, the patterns and
# the associated tag to their associated list
for intent in data["intents"]:
    for pattern in intent["pattern"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # add the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])
# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

In [10]:
# list for training data
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [11]:
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200
# the deep learning model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Metal device set to: Apple M1 Pro
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               68992     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 27)                1755      
                                                                 
Total params: 79,003
Trainable params: 79,003
Non-trainable params: 0
_________________________________________________________________
None


2022-12-03 16:40:25.848999: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-03 16:40:25.850046: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/200


2022-12-03 16:40:26.070102: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


 1/22 [>.............................] - ETA: 10s - loss: 3.3089 - accuracy: 0.0312

2022-12-03 16:40:26.374645: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


22/22 [==============================] - 1s 16ms/step - loss: 3.1842 - accuracy: 0.1068
Epoch 2/200
22/22 [==============================] - 0s 11ms/step - loss: 2.2602 - accuracy: 0.3694
Epoch 3/200
22/22 [==============================] - 0s 11ms/step - loss: 1.5443 - accuracy: 0.5608
Epoch 4/200
22/22 [==============================] - 0s 10ms/step - loss: 1.0388 - accuracy: 0.7047
Epoch 5/200
22/22 [==============================] - 0s 10ms/step - loss: 0.7384 - accuracy: 0.7745
Epoch 6/200
22/22 [==============================] - 0s 11ms/step - loss: 0.5901 - accuracy: 0.8220
Epoch 7/200
22/22 [==============================] - 0s 10ms/step - loss: 0.4619 - accuracy: 0.8605
Epoch 8/200
22/22 [==============================] - 0s 10ms/step - loss: 0.4830 - accuracy: 0.8546
Epoch 9/200
22/22 [==============================] - 0s 10ms/step - loss: 0.3129 - accuracy: 0.9110
Epoch 10/200
22/22 [==============================] - 0s 10ms/step - loss: 0.3443 - accuracy: 0.8887
Epoch 11/20

22/22 [==============================] - 0s 10ms/step - loss: 0.1651 - accuracy: 0.9629
Epoch 83/200
22/22 [==============================] - 0s 10ms/step - loss: 0.1230 - accuracy: 0.9644
Epoch 84/200
22/22 [==============================] - 0s 10ms/step - loss: 0.1579 - accuracy: 0.9614
Epoch 85/200
22/22 [==============================] - 0s 10ms/step - loss: 0.1338 - accuracy: 0.9674
Epoch 86/200
22/22 [==============================] - 0s 10ms/step - loss: 0.0823 - accuracy: 0.9659
Epoch 87/200
22/22 [==============================] - 0s 10ms/step - loss: 0.2087 - accuracy: 0.9599
Epoch 88/200
22/22 [==============================] - 0s 10ms/step - loss: 0.1329 - accuracy: 0.9703
Epoch 89/200
22/22 [==============================] - 0s 10ms/step - loss: 0.1084 - accuracy: 0.9703
Epoch 90/200
22/22 [==============================] - 0s 10ms/step - loss: 0.1088 - accuracy: 0.9674
Epoch 91/200
22/22 [==============================] - 0s 10ms/step - loss: 0.1004 - accuracy: 0.9614
Epo

22/22 [==============================] - 0s 11ms/step - loss: 0.0797 - accuracy: 0.9792
Epoch 163/200
22/22 [==============================] - 0s 11ms/step - loss: 0.0747 - accuracy: 0.9837
Epoch 164/200
22/22 [==============================] - 0s 10ms/step - loss: 0.1582 - accuracy: 0.9733
Epoch 165/200
22/22 [==============================] - 0s 10ms/step - loss: 0.1608 - accuracy: 0.9703
Epoch 166/200
22/22 [==============================] - 0s 11ms/step - loss: 0.1131 - accuracy: 0.9792
Epoch 167/200
22/22 [==============================] - 0s 11ms/step - loss: 0.1135 - accuracy: 0.9807
Epoch 168/200
22/22 [==============================] - 0s 11ms/step - loss: 0.0996 - accuracy: 0.9822
Epoch 169/200
22/22 [==============================] - 0s 10ms/step - loss: 0.0642 - accuracy: 0.9807
Epoch 170/200
22/22 [==============================] - 0s 10ms/step - loss: 0.1137 - accuracy: 0.9748
Epoch 171/200
22/22 [==============================] - 0s 11ms/step - loss: 0.1682 - accuracy: 0

In [12]:
def clean_text(text): 
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

def bag_of_words(text, vocab): 
    tokens = clean_text(text)
    bow = [0] * len(vocab)
    for w in tokens: 
        for idx, word in enumerate(vocab):
            if word == w: 
                bow[idx] = 1
    return np.array(bow)

def pred_class(text, vocab, labels): 
    bow = bag_of_words(text, vocab)
    result = model.predict(np.array([bow]),verbose=0)[0]
    thresh = 0.2
    y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list[0]



In [ ]:
# running the chatbot
print(color.GREEN+"ROBO: Hello. If you want to exit, type Bye!"+color.END)
while True:
    message = input("Me: ")
    if message.lower() == 'bye':
        print(color.GREEN+"ROBO: Bye! take care.."+color.END)
        break
    
    intents = pred_class(message, words, classes)
    print(intents)

ROBO: Hello. If you want to exit, type Bye!
Me: I need help


2022-12-03 16:41:26.027606: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


help
Me: can you help me
help
Me: thanks
thanks
Me: I feel bad
User-agree
Me: I feel sad
sad
Me: I am so sad
sad
Me: I feel so useless
worthless
Me:  
greeting
Me:  
greeting
Me: 
greeting
Me: ex
greeting
Me: dadadada
greeting
Me: I don't want to talk
not-talking
Me: I am fucking angry
Angry
